# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Programa

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

Esses valores podem ser atribuídos na função `main`, não é necessária leitura pelo teclado. Entretanto, o programa deve ser genérico, ou seja, funcionará para qualquer outro valor atribuído às 3 variáveis.

In [1]:
#include <stdio.h>

int main()
{
    float S = 200; // valor da primeira parcela
    int N = 5; // número de parcelas
    float J = 0.01; // taxa de juros mensal
    
    int i;
    for (i = 0; i < N; i++){
        printf("%.2f\n", S);
        S = S * (1 + J);
    }
    return 0;
}

200.00
202.00
204.02
206.06
208.12


## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de cada parcela X do empréstimo.

Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa em que está inserida. Considere que esta função poderia ser eventualmente reusada em outro programa e deveria ser possível fazê-lo sem que ela dependa de nada do programa em que irá se inserir.

A função deve apenas calcular uma única parcela em cada chamada, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [2]:
#include <stdio.h>

float proximaParcela(float S, float J)
{   
    // calcula a proxima parcela, dada a parcela atual e a taxa de juros
    // S: valor da parcela atual
    // J: taxa de juros mensal
    
    return S * (1 + J);
}

int main()
{
    float S = 200; // valor da primeira parcela
    int N = 5;
    float J = 0.01;
    
    int i;
    for (i = 0; i < N; i++){
        printf("%.2f\n", S);
        S = proximaParcela(S, J);
    }
    return 0;
}

200.00
202.00
204.02
206.06
208.12


## Exercício Parte 3 - Minimizando os Parâmetros

Na solução anterior, você deve ter usado vários parâmetros para a comunicação entre o `main` com a função `proximaParcela`, evitando usar variáveis globais. Modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar todos os valores que estejam relacionados a um empréstimo.

In [3]:


#include <stdio.h>
#include <stdlib.h>

typedef struct _emprestimo {
    float S; // valor da parcela atual
    float J; // taxa de juros mensal
} Emprestimo;

float proximaParcela(Emprestimo emp)
{
    // calcula a proxima parcela de um emprestimo
    // emp: emprestimo
    
    return emp.S * (1 + emp.J);
}

int main(){
    int N = 5; // número de parcelas
    
    Emprestimo emp;
    emp.S = 200;
    emp.J = 0.01;
    
    int i;
    for (i = 0; i < N; i++){
        printf("%f\n", emp.S);
        emp.S = proximaParcela(emp);
    }
    return 0;
}

200.000000
202.000000
204.020004
206.060196
208.120789


## Exercício Parte 4 - Mantendo o Estado

A função que você codificou na solução anterior provavelmente não é capaz de manter o controle do que chamaremos de **estado do empréstimo**, ou seja, dados de qual a parcela corrente ou da última parcela calculada.

Modifique a função (módulo) `proximaParcela` acima de modo que ela seja capaz de guardar o **estado do empréstimo** por sua conta, sem que o `main` precise ficar informando este estado por parâmetros (por exemplo). Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência das funções com o programa em que ela está inserida.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente. O ideal é que o main não tenha acesso ao modo como as funções vão representar o **estado do empréstimo**, para que toda a lógica do empréstimo fique dentro das funções.

In [4]:
#include <stdio.h>
#include <stdlib.h>

typedef struct _emprestimo {
    float S; // valor da parcela atual
    float J; // taxa de juros mensal
} Emprestimo;

typedef struct _lista
{
    // lista de parcelas
    float parcela;
    struct _lista *prox;
} Lista;

Lista *inserir_elemento(float parcela, Lista *ls)
{
    // insere novo elemento no início de uma lista de parcelas
    Lista *nova_lista = malloc(sizeof(Lista));
    nova_lista->parcela = parcela;
    nova_lista->prox = ls;
    return nova_lista;
}

Lista *acessar_elemento(int indice, int prestacoes, Lista *ls)
{
    // acessa em uma lista o elemento no indice dado
    // NOTE: o índice nesse caso começa do 1, não do 0
    // WARNING: essa função assume que o elemento a ser acessado existe
    
    int i;
    for (i = prestacoes; i != indice; i--){
        ls = ls->prox;
    }
    return ls;
}

void free_lista(Lista *ls)
{
    // libera a memória ocupada por uma lista
    if (ls) {
        free_lista(ls->prox);
        free(ls);
    }
}

Lista *criar_lista()
{
    // cria uma lista vazia
    return NULL;
}

float proximaParcela(Emprestimo emp)
{
    // calcula a proxima parcela de um emprestimo
    // emp: emprestimo
    
    return emp.S * (1 + emp.J);
}

int main()
{
    int N = 5; // número de parcelas
    
    Emprestimo emp;
    emp.S = 200;
    emp.J = 0.01;
    
    Lista *ls = criar_lista();
    
    int i;
    for (i = 0; i < N; i++){
        printf("%.2f\n", emp.S);
        ls = inserir_elemento(emp.S, ls);
        emp.S = proximaParcela(emp);
    }
    
    // mostrar que o estado do empréstimo está sendo devidamente armazenado
    printf("\nAcesso por índice:\n");
    for (i = 1; i <= N; i++)
        printf("%.2f\n", acessar_elemento(i, N, ls)->parcela);    
    free_lista(ls);
    return 0;
}

200.00
202.00
204.02
206.06
208.12

Acesso por índice:
200.00
202.00
204.02
206.06
208.12


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada **estado de empréstimo** separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, as funções de cálculo do empréstimo ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que comporte até 100 empréstimo e, cada novo empréstimo, só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo (até o limite de 100).

O seu programa deve apresentar o exemplo acima como teste.

In [23]:
#include <stdio.h>
#include <stdlib.h>

#define MAX_EMPRESTIMOS 100

typedef struct _lista
{
    // lista de parcelas
    float parcela;
    struct _lista *prox;
} Lista;

Lista *inserir_elemento(float parcela, Lista *ls)
{
    // insere novo elemento no início de uma lista de parcelas
    Lista *nova_lista = malloc(sizeof(Lista));
    nova_lista->parcela = parcela;
    nova_lista->prox = ls;
    return nova_lista;
}

Lista *acessar_elemento(int indice, int prestacoes, Lista *ls)
{
    // acessa em uma lista o elemento no indice dado
    // NOTE: o índice nesse caso começa do 1, não do 0
    // WARNING: essa função assume que o elemento a ser acessado existe
    
    int i;
    for (i = prestacoes; i != indice; i--){
        ls = ls->prox;
    }
    return ls;
}

void free_lista(Lista *ls)
{
    // libera a memória ocupada por uma lista
    if (ls) {
        free_lista(ls->prox);
        free(ls);
    }
}

Lista *criar_lista()
{
    // cria uma lista vazia
    return NULL;
}

typedef struct _emprestimo {
    float S;   // valor da parcela atual
    float J;   // taxa de juros mensal
    int N;     // número de prestações
    Lista *ls; // lista de parcelas
} Emprestimo;


float proximaParcela(Emprestimo emp)
{
    // calcula a proxima parcela de um emprestimo
    // emp: emprestimo
    
    return emp.S * (1 + emp.J);
}

int maxPrestacoesEmprestimos(Emprestimo emprestimos[], int numero_de_emprestimos)
{
    // retorna o máximo número de prestações entre todos os empréstimos no vetor
    int i;
    int max = 0;
    for (i = 1; i <= numero_de_emprestimos; i++){
        if (emprestimos[i].N > max)
            max = emprestimos[i].N;
    }
    return max;
}

int todosEmprestimosPagos(Emprestimo emprestimos[], int numero_de_emprestimos, int mes)
{
    // verifica se todos os emprestimos no vetor foram pagos ate o mes atual
    return mes > maxPrestacoesEmprestimos(emprestimos, numero_de_emprestimos);
}

int main()
{
    Emprestimo emp1;
    emp1.S = 200;
    emp1.J = 0.01;
    emp1.N = 5;
    emp1.ls = criar_lista();
    
    Emprestimo emp2;
    emp2.S = 500;
    emp2.J = 0.02;
    emp2.N = 7;
    emp2.ls = criar_lista();
    
    // caso fôssemos expandir esse programa para receber input do usuário,
    // a variável abaixo precisaria ser atualizada a cada novo empréstimo 
    // enviado pelo usuário. No entanto, como estamos trabalhando com um
    // caso simplificado vamos simplesmente usar uma constante
    const int numero_de_emprestimos = 2;
    
    Emprestimo emprestimos[MAX_EMPRESTIMOS + 1];
    // deixar empréstimo 1 no índice 1, e assim sucessivamente, por clareza
    emprestimos[1] = emp1;
    emprestimos[2] = emp2;
    
    int i, mes = 1;
    while(!todosEmprestimosPagos(emprestimos, numero_de_emprestimos, mes)){
        for (i = 1; i <= numero_de_emprestimos; i++){
            if (emprestimos[i].N <= mes){
                printf("%.2f\n", emprestimos[i].S);
                emprestimos[i].S = proximaParcela(emprestimos[i]);
            }
        }
        mes++;
    }
    
    
    free_lista(emp1.ls);
    free_lista(emp2.ls);
}

200.00
202.00
204.02
500.00
